## All Techniques of Hyper Parameter Optimization

<ol>
    <li>GridSearchCV</li>
    <li>RandomizedSearchCV</li>
    <li>Bayesian Optimization- Automate HyperParameter Tuning (Hyperopt)</li>
    <li>Sequential Model Based Optimization(Tuning a scikit-learn estimator with skopt)</li>
    <li>Optuna- Automate Hyperparameter Tuning</li>
    <li>Genetic Algorithms (TPOT Classifier)</li>
</ol>    


### We basically use hyper parameter optimization to increase the accuracy by selecting the best possible parameters

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
df=pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np
df['Glucose'] = np.where(df['Glucose']==0, df['Glucose'].median(), df['Glucose'])
df['Insulin'] = np.where(df['Insulin']==0, df['Insulin'].median(), df['Insulin'])
df['SkinThickness'] = np.where(df['SkinThickness']==0, df['SkinThickness'].median(), df['SkinThickness'])

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35.0,30.5,33.6,0.627,50,1
1,1,85.0,66,29.0,30.5,26.6,0.351,31,0
2,8,183.0,64,23.0,30.5,23.3,0.672,32,1
3,1,89.0,66,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40,35.0,168.0,43.1,2.288,33,1


In [4]:
# Splitting into Independent and dependent features
X = df.drop('Outcome',axis=1)
y = df['Outcome']

In [5]:
## Train_test_split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( 
                                                    X, y, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

In [6]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators = 10).fit(X_train, y_train)
prediction = rf_classifier.predict(X_test)

In [7]:
y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [8]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[95 12]
 [22 25]]
0.7792207792207793
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       107
           1       0.68      0.53      0.60        47

    accuracy                           0.78       154
   macro avg       0.74      0.71      0.72       154
weighted avg       0.77      0.78      0.77       154



The main parameters used by a Random Forest Classifier are:

- criterion = the function used to evaluate the quality of a split.
- max_depth = maximum number of levels allowed in each tree.
- max_features = maximum number of features considered when splitting a node.
- min_samples_leaf = minimum number of samples which can be stored in a tree leaf.
- min_samples_split = minimum number of samples necessary in a node to cause node splitting.
- n_estimators = number of trees in the ensamble.

In [9]:
### Manual Hyperparameter Tuning i.e I randomly selected any parameters based on my mood
model=RandomForestClassifier(n_estimators=300,criterion='entropy',
                             max_features='sqrt',min_samples_leaf=10,random_state=100).fit(X_train,y_train)
predictions=model.predict(X_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

[[97 10]
 [17 30]]
0.8246753246753247
              precision    recall  f1-score   support

           0       0.85      0.91      0.88       107
           1       0.75      0.64      0.69        47

    accuracy                           0.82       154
   macro avg       0.80      0.77      0.78       154
weighted avg       0.82      0.82      0.82       154



### !!! IMPORTANT !!!

- RandomizedSearchCv randomly search the parameters and gives the best possible outputs.
- GridSearchCv goes through every possible value and tries to find the best solution

- We should always perform RandomizedSearchCV first to narrow down the possibilities and then apply GridSearchCv based on the outcome of RandomizedSearchCV

## RandomizedSearchCv

In [10]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [11]:
rf=RandomForestClassifier()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,
                               random_state=100,n_jobs=-1)
### fit the randomized model
rf_randomcv.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 5, 10, 14],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [12]:
rf_randomcv.best_params_

{'n_estimators': 600,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 780,
 'criterion': 'gini'}

In [13]:
rf_randomcv.best_estimator_

RandomForestClassifier(max_depth=780, max_features='sqrt', min_samples_split=5,
                       n_estimators=600)

In [14]:

best_random_grid=rf_randomcv.best_estimator_

In [15]:
from sklearn.metrics import accuracy_score
y_pred=best_random_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: {}".format(classification_report(y_test,y_pred)))

[[95 12]
 [15 32]]
Accuracy Score 0.8246753246753247
Classification report:               precision    recall  f1-score   support

           0       0.86      0.89      0.88       107
           1       0.73      0.68      0.70        47

    accuracy                           0.82       154
   macro avg       0.80      0.78      0.79       154
weighted avg       0.82      0.82      0.82       154



In [16]:
# WE CAN FIND INCREASE IN THE ACCURACY 
# SINCE IT IS A SMALL DATASET WE CAN'T FIND MUCH DIFFERENCE

## GridSearchCV

In [17]:
rf_randomcv.best_params_

{'n_estimators': 600,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 780,
 'criterion': 'gini'}

In [18]:
# Now we will input parameters close to these values for gridSearch in order to narrow down the search

from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': [rf_randomcv.best_params_['criterion']],
    'max_depth': [rf_randomcv.best_params_['max_depth']],
    'max_features': [rf_randomcv.best_params_['max_features']],
    'min_samples_leaf': [rf_randomcv.best_params_['min_samples_leaf'], 
                         rf_randomcv.best_params_['min_samples_leaf']+2, 
                         rf_randomcv.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rf_randomcv.best_params_['min_samples_split'] - 2,
                          rf_randomcv.best_params_['min_samples_split'] - 1,
                          rf_randomcv.best_params_['min_samples_split'], 
                          rf_randomcv.best_params_['min_samples_split'] +1,
                          rf_randomcv.best_params_['min_samples_split'] + 2],
    'n_estimators': [rf_randomcv.best_params_['n_estimators'] - 200, rf_randomcv.best_params_['n_estimators'] - 100, 
                     rf_randomcv.best_params_['n_estimators'], 
                     rf_randomcv.best_params_['n_estimators'] + 100, rf_randomcv.best_params_['n_estimators'] + 200]
}

print(param_grid)

{'criterion': ['gini'], 'max_depth': [780], 'max_features': ['sqrt'], 'min_samples_leaf': [1, 3, 5], 'min_samples_split': [3, 4, 5, 6, 7], 'n_estimators': [400, 500, 600, 700, 800]}


In [19]:
#### Fit the grid_search to the data
# Here we do not give iterations as the parameter bcoz gridsearch will do all possible iterations
# in this case there will be 1*1*1*3*5*5
rf=RandomForestClassifier()
grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 75 candidates, totalling 750 fits


GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini'], 'max_depth': [780],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 3, 5],
                         'min_samples_split': [3, 4, 5, 6, 7],
                         'n_estimators': [400, 500, 600, 700, 800]},
             verbose=2)

In [20]:
grid_search.best_estimator_

RandomForestClassifier(max_depth=780, max_features='sqrt', min_samples_split=4,
                       n_estimators=800)

In [21]:
best_grid=grid_search.best_estimator_

In [22]:
best_grid

RandomForestClassifier(max_depth=780, max_features='sqrt', min_samples_split=4,
                       n_estimators=800)

In [23]:
y_pred=best_grid.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("Classification report: {}".format(classification_report(y_test,y_pred)))


[[94 13]
 [15 32]]
Accuracy Score 0.8181818181818182
Classification report:               precision    recall  f1-score   support

           0       0.86      0.88      0.87       107
           1       0.71      0.68      0.70        47

    accuracy                           0.82       154
   macro avg       0.79      0.78      0.78       154
weighted avg       0.82      0.82      0.82       154



## Automated Hyperparameter Tuning

Automated Hyperparameter Tuning can be done by using techniques such as

- Bayesian Optimization
- Gradient Descent
- Evolutionary Algorithms

### Bayesian Optimization

Bayesian optimization uses probability to find the minimum of a function. The final aim is to find the input value to a function which can gives us the lowest possible output value.It usually performs better than random,grid and manual search providing better performance in the testing phase and reduced optimization time. In Hyperopt, Bayesian Optimization can be implemented giving 3 three main parameters to the function fmin.

- Objective Function = defines the loss function to minimize.
- Domain Space = defines the range of input values to test (in Bayesian Optimization this space creates a probability distribution for each of the used Hyperparameters).
- Optimization Algorithm = defines the search algorithm to use to select the best input values to use in each new iteration.

In [24]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [25]:
space = {'criterion': hp.choice('criterion', ['entropy', 'gini']), #.choice for selecting from the options
        'max_depth': hp.quniform('max_depth', 10, 1200, 10), # .quniform for selecting integers between these 2 numbers 
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5), # .uniform for selecting floating values
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.choice('n_estimators', [10, 50, 300, 750, 1200,1300,1500])
    }

In [26]:
space

{'criterion': <hyperopt.pyll.base.Apply at 0x15435f8>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x15436d0>,
 'max_features': <hyperopt.pyll.base.Apply at 0x1543760>,
 'min_samples_leaf': <hyperopt.pyll.base.Apply at 0x1543850>,
 'min_samples_split': <hyperopt.pyll.base.Apply at 0x15438e0>,
 'n_estimators': <hyperopt.pyll.base.Apply at 0x1543958>}

In [27]:
def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = space['max_depth'],
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = space['n_estimators'], 
                                 )
    
    accuracy = cross_val_score(model, X_train, y_train, cv = 5).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK }

In [28]:
from sklearn.model_selection import cross_val_score
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 80,
            trials= trials)
best

100%|███████████████████████████████████████████████| 80/80 [22:21<00:00, 16.77s/trial, best loss: -0.7671064907370384]


{'criterion': 0,
 'max_depth': 60.0,
 'max_features': 2,
 'min_samples_leaf': 0.09750784459926931,
 'min_samples_split': 0.059791037594904645,
 'n_estimators': 3}

In [29]:
crit = {0: 'entropy', 1: 'gini'}
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 10, 1: 50, 2: 300, 3: 750, 4: 1200,5:1300,6:1500}


print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

entropy
log2
750


In [30]:
trainedforest = RandomForestClassifier(criterion = crit[best['criterion']], max_depth = best['max_depth'], 
                                       max_features = feat[best['max_features']], 
                                       min_samples_leaf = best['min_samples_leaf'], 
                                       min_samples_split = best['min_samples_split'], 
                                       n_estimators = est[best['n_estimators']]).fit(X_train,y_train)
predictionforest = trainedforest.predict(X_test)
print(confusion_matrix(y_test,predictionforest))
print(accuracy_score(y_test,predictionforest))
print(classification_report(y_test,predictionforest))
acc5 = accuracy_score(y_test,predictionforest)

[[96 11]
 [25 22]]
0.7662337662337663
              precision    recall  f1-score   support

           0       0.79      0.90      0.84       107
           1       0.67      0.47      0.55        47

    accuracy                           0.77       154
   macro avg       0.73      0.68      0.70       154
weighted avg       0.75      0.77      0.75       154



## Genetic Algorithms

Genetic Algorithms tries to apply natural selection mechanisms to Machine Learning contexts.

Let's immagine we create a population of N Machine Learning models with some predifined Hyperparameters. We can then calculate the accuracy of each model and decide to keep just half of the models (the ones that performs best). We can now generate some offsprings having similar Hyperparameters to the ones of the best models so that go get again a population of N models. At this point we can again caltulate the accuracy of each model and repeate the cycle for a defined number of generations. In this way, just the best models will survive at the end of the process.

In [32]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(param)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [34]:
! pip install tpot

    ERROR: Command errored out with exit status 1:
     command: 'c:\users\mrudu\appdata\local\programs\python\python38-32\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Mrudu\\AppData\\Local\\Temp\\pip-install-oh3yo8tv\\xgboost_bc3df27d2e1244f8a22f8087edd6cd78\\setup.py'"'"'; __file__='"'"'C:\\Users\\Mrudu\\AppData\\Local\\Temp\\pip-install-oh3yo8tv\\xgboost_bc3df27d2e1244f8a22f8087edd6cd78\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record 'C:\Users\Mrudu\AppData\Local\Temp\pip-record-v_eb983r\install-record.txt' --single-version-externally-managed --compile --install-headers 'c:\users\mrudu\appdata\local\programs\python\python38-32\Include\xgboost'
         cwd: C:\Users\Mrudu\AppData\Local\Temp\pip-inst


Using legacy 'setup.py install' for deap, since package 'wheel' is not installed.
Using legacy 'setup.py install' for stopit, since package 'wheel' is not installed.
Using legacy 'setup.py install' for xgboost, since package 'wheel' is not installed.
    Running setup.py install for xgboost: started
    Running setup.py install for xgboost: finished with status 'error'



    creating build\lib.win32-3.8\xgboost\dmlc-core\include\dmlc
    copying xgboost\dmlc-core\include\dmlc\any.h -> build\lib.win32-3.8\xgboost\dmlc-core\include\dmlc
    copying xgboost\dmlc-core\include\dmlc\array_view.h -> build\lib.win32-3.8\xgboost\dmlc-core\include\dmlc
    copying xgboost\dmlc-core\include\dmlc\base.h -> build\lib.win32-3.8\xgboost\dmlc-core\include\dmlc
    copying xgboost\dmlc-core\include\dmlc\blockingconcurrentqueue.h -> build\lib.win32-3.8\xgboost\dmlc-core\include\dmlc
    copying xgboost\dmlc-core\include\dmlc\build_config_default.h -> build\lib.win32-3.8\xgboost\dmlc-core\include\dmlc
    copying xgboost\dmlc-core\include\dmlc\common.h -> build\lib.win32-3.8\xgboost\dmlc-core\include\dmlc
    copying xgboost\dmlc-core\include\dmlc\concurrency.h -> build\lib.win32-3.8\xgboost\dmlc-core\include\dmlc
    copying xgboost\dmlc-core\include\dmlc\concurrentqueue.h -> build\lib.win32-3.8\xgboost\dmlc-core\include\dmlc
    copying xgboost\dmlc-core\include\dmlc\

    copying xgboost\src\tree\updater_sync.cc -> build\lib.win32-3.8\xgboost\src\tree
    creating build\lib.win32-3.8\xgboost\src\tree\gpu_hist
    copying xgboost\src\tree\gpu_hist\driver.cuh -> build\lib.win32-3.8\xgboost\src\tree\gpu_hist
    copying xgboost\src\tree\gpu_hist\evaluate_splits.cu -> build\lib.win32-3.8\xgboost\src\tree\gpu_hist
    copying xgboost\src\tree\gpu_hist\evaluate_splits.cuh -> build\lib.win32-3.8\xgboost\src\tree\gpu_hist
    copying xgboost\src\tree\gpu_hist\feature_groups.cu -> build\lib.win32-3.8\xgboost\src\tree\gpu_hist
    copying xgboost\src\tree\gpu_hist\feature_groups.cuh -> build\lib.win32-3.8\xgboost\src\tree\gpu_hist
    copying xgboost\src\tree\gpu_hist\gradient_based_sampler.cu -> build\lib.win32-3.8\xgboost\src\tree\gpu_hist
    copying xgboost\src\tree\gpu_hist\gradient_based_sampler.cuh -> build\lib.win32-3.8\xgboost\src\tree\gpu_hist
    copying xgboost\src\tree\gpu_hist\histogram.cu -> build\lib.win32-3.8\xgboost\src\tree\gpu_hist
    cop

In [35]:
from tpot import TPOTClassifier


tpot_classifier = TPOTClassifier(generations= 5, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestClassifier': param}, 
                                 cv = 4, scoring = 'accuracy')
tpot_classifier.fit(X_train,y_train)

ModuleNotFoundError: No module named 'tpot'

In [ ]:
accuracy = tpot_classifier.score(X_test, y_test)
print(accuracy)

## Optimize hyperparameters of the model using Optuna

The hyperparameters of the above algorithm are n_estimators and max_depth for which we can try different values to see if the model accuracy can be improved. The objective function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [36]:
import optuna
import sklearn.svm
def objective(trial):

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 200, 2000,10)
        max_depth = int(trial.suggest_float('max_depth', 10, 100, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf,X_train,y_train, n_jobs=-1, cv=3).mean()

In [37]:
study = optuna.create_study(direction='maximize') # to maximize the accuracy
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2021-08-25 18:47:25,838] A new study created in memory with name: no-name-5d6dd5a6-ed19-44bd-9b40-10275ca87e4a
[I 2021-08-25 18:47:30,654] Trial 0 finished with value: 0.7491790212019768 and parameters: {'classifier': 'RandomForest', 'n_estimators': 780, 'max_depth': 80.7094988008118}. Best is trial 0 with value: 0.7491790212019768.
[I 2021-08-25 18:47:32,485] Trial 1 finished with value: 0.640068547744301 and parameters: {'classifier': 'SVC', 'svc_c': 34198.45306178281}. Best is trial 0 with value: 0.7491790212019768.
[I 2021-08-25 18:47:42,542] Trial 2 finished with value: 0.7459269886816515 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1340, 'max_depth': 40.164961770535314}. Best is trial 0 with value: 0.7491790212019768.
[I 2021-08-25 18:47:44,679] Trial 3 finished with value: 0.7361310377809661 and parameters: {'classifier': 'RandomForest', 'n_estimators': 320, 'max_depth': 42.48196762770072}. Best is trial 0 with value: 0.7491790212019768.
[I 2021-08-25 18:47:

[I 2021-08-25 18:50:50,397] Trial 35 finished with value: 0.640068547744301 and parameters: {'classifier': 'SVC', 'svc_c': 0.003817480181671908}. Best is trial 8 with value: 0.7589430894308943.
[I 2021-08-25 18:50:54,162] Trial 36 finished with value: 0.7475450342738722 and parameters: {'classifier': 'RandomForest', 'n_estimators': 620, 'max_depth': 18.734422310136146}. Best is trial 8 with value: 0.7589430894308943.
[I 2021-08-25 18:50:56,429] Trial 37 finished with value: 0.7459349593495935 and parameters: {'classifier': 'RandomForest', 'n_estimators': 360, 'max_depth': 29.083881758044477}. Best is trial 8 with value: 0.7589430894308943.
[I 2021-08-25 18:50:56,570] Trial 38 finished with value: 0.640068547744301 and parameters: {'classifier': 'SVC', 'svc_c': 4.6467711105918783e-08}. Best is trial 8 with value: 0.7589430894308943.
[I 2021-08-25 18:51:01,181] Trial 39 finished with value: 0.7491790212019768 and parameters: {'classifier': 'RandomForest', 'n_estimators': 790, 'max_depth'

[I 2021-08-25 19:00:44,639] Trial 70 finished with value: 0.7475450342738722 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1440, 'max_depth': 41.71487371537739}. Best is trial 8 with value: 0.7589430894308943.
[I 2021-08-25 19:00:48,147] Trial 71 finished with value: 0.7459190180137095 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1130, 'max_depth': 23.185915240589225}. Best is trial 8 with value: 0.7589430894308943.
[I 2021-08-25 19:00:52,656] Trial 72 finished with value: 0.7524310537223019 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1670, 'max_depth': 10.076744528089689}. Best is trial 8 with value: 0.7589430894308943.
[I 2021-08-25 19:00:54,110] Trial 73 finished with value: 0.7459190180137095 and parameters: {'classifier': 'RandomForest', 'n_estimators': 540, 'max_depth': 19.001900128033405}. Best is trial 8 with value: 0.7589430894308943.
[I 2021-08-25 19:00:58,204] Trial 74 finished with value: 0.7508130081300813 and parameter

Accuracy: 0.7589430894308943
Best hyperparameters: {'classifier': 'RandomForest', 'n_estimators': 480, 'max_depth': 24.6495304256133}
